In [4]:
# Processing pipeline to turn generated data into a format that is easy to read into tensorflow: TFRecord files.
# These files allow for fast read and write to Tensorflow architectures, and are preferred using the new Dataset API. 
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import imageio
import numpy
import glob
from skimage.transform import resize
import sys

# The documentation for this code is somewhat spotty, and has been cobbled together from a variety of sources:
# https://medium.com/@WuStangDan/step-by-step-tensorflow-object-detection-api-tutorial-part-2-converting-dataset-to-tfrecord-47f24be9248d
# http://machinelearninguru.com/deep_learning/tensorflow/basics/tfrecord/tfrecord.html
# https://www.skcript.com/svr/why-every-tensorflow-developer-should-know-about-tfrecord/
# tensorflow/inception/inception/data/build_image_Data.py

# Once within the format of tfrecord files, it is easier to combine datasets, to split them, etc, so we just treat
# these on a video by video basis. 


In [5]:
## Designate helper function to define features for examples more easily
def _int64_feature_(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature_(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _full_example_(videoname,reader,i):
    img_0 = reader.get_data(i)
    img_0_ds = resize(img_0,(128,128))

    features = {'video': _bytes_feature_(tf.compat.as_bytes((videoname))),
                'frame': _int64_feature_(i),
                'image': _bytes_feature_(tf.compat.as_bytes(img_0_ds.tostring())),
                }
    example = tf.train.Example(features = tf.train.Features(feature = features))
    return example

In [6]:
# We iterate through all directories
videos = glob.glob('datadirectory/toydynamics_nograv/*.mp4')
for videoname in videos:
    print('working on video: ' +videoname)
    # Choose a video, and load it
    video = imageio.get_reader(videoname,'ffmpeg')
    # We want to open a corresponding tfRecords file
    datafile_name = videoname.split('.mp4')[0]+'train.tfrecords'
    writer = tf.python_io.TFRecordWriter(datafile_name)
    nb_frames = video.get_length()
    for i in range(nb_frames):
        # Iterate through frames
        example = _full_example_(videoname,video,i)
        writer.write(example.SerializeToString())
    writer.close()
    sys.stdout.flush()
    

working on video: datadirectory/toydynamics_nograv/Video_ball_color.mp4


/Users/taigaabe/anaconda3/envs/constrainedgen/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


working on video: datadirectory/toydynamics_nograv/Video_ball_color_noback.mp4
working on video: datadirectory/toydynamics_nograv/Video_ball_color_big.mp4
